# Setup

In [59]:
import numpy as np
import pandas as pd
import sklearn
import pyreadstat   # for reading Stata files

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

#from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.tree import export_graphviz
from sklearn.model_selection import RandomizedSearchCV

import matplotlib.pyplot as plt
import joblib                                              # running computationally intensive tasks in parallel
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
import sklearn.metrics

import statsmodels.api as sm

# Load Data

In [15]:
# Set working directory located in 'data' folder
import os
os.chdir('data')

df1_raw, _ = pyreadstat.read_dta("rf_coh1_27.dta")
df2_raw, _ = pyreadstat.read_dta("rf_coh2_27.dta")
df3_raw, _ = pyreadstat.read_dta("rf_coh2_37.dta")
df4_raw, _ = pyreadstat.read_dta("rf_coh3_37.dta")

In [19]:
# Define preprocessing function
def preprocess_dataset(file_path):
    # Read Stata datafile
    df_raw, meta = pyreadstat.read_dta(file_path)
    
    # Operationalization
    df_processed = df_raw.copy()
    df_processed['pincnet_log'] = np.log(df_processed['pincnet'].replace(0, 1))
    df_processed['hhincnet_log'] = np.log(df_processed['hhincnet'].replace(0, 1))
    df_processed['pinc_decile'] = pd.qcut(df_processed['pincnet_log'], 10, labels=False, duplicates='drop') + 1
    df_processed['hhincnet_decile'] = pd.qcut(df_processed['hhincnet_log'], 10, labels=False, duplicates='drop') + 1
    df_processed['bmi_category'] = pd.cut(df_processed['bmi'], bins=[-np.inf, 18.5, 25, 30, np.inf], labels=["Underweight", "Normal", "Overweight", "Obese"])
    df_processed['relstat'] = df_processed['relstat'].map({
        1: "Single", 5: "Single", 6: "Single", 9: "Single",
        2: "LAT", 7: "LAT", 10: "LAT",
        3: "Cohabiting", 8: "Cohabiting", 11: "Cohabiting",
        4: "Married"
    })
    df_processed['nkids'] = df_processed['nkids'].apply(lambda x: x if x < 3 else 3)
    df_processed['empl'] = df_processed['lfs'].map({
        1: "Education/not working", 2: "Education/not working", 3: "Education/not working", 4: "Education/not working",
        5: "Education/not working", 6: "Education/not working", 7: "Education/not working", 8: "Education/not working",
        9: "Full-time or self-employment", 12: "Full-time or self-employment", 13: "Full-time or self-employment",
        10: "Marginal or part-time employment", 11: "Marginal or part-time employment"
    })
    df_processed['edu'] = df_processed['isced'].map({
        1: "Basic", 2: "Basic",
        0: "Intermediate", 3: "Intermediate", 4: "Intermediate",
        5: "Advanced", 6: "Advanced", 7: "Advanced", 8: "Advanced"
    })
    df_processed['migback'] = df_processed['migstatus'].map({1: 0, 2: 1, 3: 1})
    df_processed['urban'] = df_processed['gkpol'].apply(lambda x: 0 if x in [1, 2, 3, 4] else 1)
    df_processed['sex_often'] = df_processed['sexfreq'].apply(lambda x: 1 if x >= 4 else 0)
    df_processed['denomination'] = df_processed['sd30'].map({
        7: "None", 1: "Roman Catholic", 2: "Protestant", 3: "Muslim",
        4: "Other", 5: "Other", 6: "Other"
    })
    df_processed['rel'] = df_processed['sd31'].map({
        6: "Never", 5: "Seldom", 4: "Occasionally",
        1: "Frequent", 2: "Frequent", 3: "Frequent"
    })
    df_processed['gendercomp'] = df_processed['gendercomp'].map({
        0: "No children", 1: "Equal gender composition", 
        2: "More boys", 3: "More girls"
    })
    df_processed['migstatus'] = df_processed['migstatus'].map({
        1: "No migration background", 2: "1st generation", 3: "2nd generation"
    })
    df_processed['health'] = df_processed['health'].map({
        1: "Bad", 2: "Not so good", 3: "Satisfactory",
        4: "Good", 5: "Very good"
    })
    df_processed['region'] = df_processed['region'].map({
        0: "Western Germany", 1: "Eastern Germany", 2: "Abroad"
    })
    df_processed['ethni'] = df_processed['ethni'].map({
        1: "German native", 2: "Ethnic-German", 3: "Half-German",
        4: "Turkish background", 5: "Other non-German background"
    })
    df_processed['addchild'] = df_processed['addchild'].map({
        0: "No", 1: "Yes", 2: "Unsure"
    })
    
    # Convert variables to appropriate types
    factors = ['hormon', 'hormon_iudh', 'hormon_iudnh', 'hormon_iudor', 
               'relstat', 'nkids', 'lfs', 'isced', 'migstatus', 'health', 'region', 'bula', 
               'addchild', 'gkpol', 'deadchild', 'gendercomp', 'sd30', 'sd31', 
               'abortion', 'ethni', 'empl', 'edu', 'migback', 'urban', 
               'sex_often', 'denomination', 'rel']
    numerics = ['id', 'wave', 'age', 'pincnet', 'hhincnet', 'height', 'weight', 
                'bmi', 'val1i3', 'val1i4', 'val1i5', 'sexfreq', 'extraversion', 
                'agreeableness', 'conscientiousness', 'neuroticism', 
                'openness', 'pincnet_log', 'hhincnet_log']
    
    df_processed[factors] = df_processed[factors].astype('category')
    df_processed[numerics] = df_processed[numerics].apply(pd.to_numeric, errors='coerce')
    
    # Select the desired variables
    selected_columns = ['id', 'wave', 'age', 'relstat', 'nkids', 
                        'pincnet_log', 'hhincnet_log', 'pinc_decile', 'pincnet', 'hhincnet', 'hhincnet_decile',
                        'empl', 'edu', 'ethni', 'migback', 'health', 'height', 'weight', 'bmi', 'bmi_category',
                        'val1i3', 'val1i4', 'val1i5', 'region', 'bula', 'addchild', 'urban', 'sex_often', 
                        'deadchild', 'gendercomp', 'extraversion', 'agreeableness', 'conscientiousness', 
                        'neuroticism', 'openness', 'denomination', 'rel', 'abortion', 'hormon', 'hormon_iudh', 
                        'hormon_iudnh', 'hormon_iudor']
    
    df_processed = df_processed[selected_columns]
    
    return df_processed

## Implement Preprocess

In [20]:
os.chdir("H:/oral_contraception_over_time/data") # Set working directory

# Run preprocessing step
df1_processed = preprocess_dataset("rf_coh1_27.dta")
df2_processed = preprocess_dataset("rf_coh2_27.dta")
df3_processed = preprocess_dataset("rf_coh2_37.dta")
df4_processed = preprocess_dataset("rf_coh3_37.dta")


In [22]:
df1_processed.head(3)

,id,wave,age,relstat,nkids,pincnet_log,hhincnet_log,pinc_decile,pincnet,hhincnet,...,conscientiousness,neuroticism,openness,denomination,rel,abortion,hormon,hormon_iudh,hormon_iudnh,hormon_iudor
0,1133000,11,27,Cohabiting,0,7.495542,8.294050,5.0,1800.0,4000.0,...,3.50,4.0,3.6,Protestant,Never,0.0,1.0,1.0,1.0,1.0
1,1420000,11,27,Cohabiting,0,8.188689,8.188689,8.0,3600.0,3600.0,...,4.00,2.0,3.4,Protestant,Never,0.0,1.0,1.0,1.0,1.0
2,1665000,13,27,Cohabiting,0,7.170120,7.170120,3.0,1300.0,1300.0,...,3.25,4.0,2.0,None,Never,1.0,0.0,0.0,0.0,0.0


## Create Final DF

In [23]:
# Define cleaning and selection function
def clean_select_df(df):
    selected_columns = [
        'wave', 'relstat', 'nkids', 'pinc_decile', 'hhincnet_decile',
        'empl', 'edu', 'ethni', 'health', 'bmi_category',
        'val1i3', 'val1i4', 'val1i5', 'region', 
        'addchild', 'urban', 'sex_often', 
        'extraversion', 'agreeableness', 'conscientiousness', 'neuroticism', 'openness', 
        'denomination', 'rel', 'abortion', 
        'migback', 'hormon_iudor'
    ]
    cleaned_df = df[selected_columns].dropna()
    return cleaned_df

# Apply the cleaning function to each processed dataset
df1_cleaned = clean_select_df(df1_processed)
df2_cleaned = clean_select_df(df2_processed)
df3_cleaned = clean_select_df(df3_processed)
df4_cleaned = clean_select_df(df4_processed)

In [56]:
df1_cleaned.dtypes

wave                    int64
relstat              category
nkids                category
pinc_decile           float64
hhincnet_decile       float64
empl                 category
edu                  category
ethni                category
health               category
bmi_category         category
val1i3                  int64
val1i4                  int64
val1i5                float64
region               category
addchild             category
urban                category
sex_often            category
extraversion          float64
agreeableness         float64
conscientiousness     float64
neuroticism           float64
openness              float64
denomination         category
rel                  category
abortion             category
migback              category
hormon_iudor         category
dtype: object

# Split Data

In [27]:
# Define function to split data
def split_data(df, test_size=0.2, random_state=123, n_splits=7):

    train_data, test_data = train_test_split(df, test_size=test_size, 
                                             stratify=df['hormon_iudor'], 
                                             random_state=random_state)
    skf = StratifiedKFold(n_splits=n_splits, 
                          shuffle=True, 
                          random_state=random_state)
    
    train_indices, val_indices = next(skf.split(train_data, 
                                                train_data['hormon_iudor']))
    train_folds = [(train_data.iloc[train_idx], 
                    train_data.iloc[val_idx]) 
                    for train_idx, val_idx in skf.split(train_data, train_data['hormon_iudor'])]
    
    return train_data.iloc[train_indices], train_folds, test_data

# Split the data into training, validation, and test data
df1_train, df1_folds, df1_test = split_data(df1_cleaned)
df2_train, df2_folds, df2_test = split_data(df2_cleaned)
df3_train, df3_folds, df3_test = split_data(df3_cleaned)
df4_train, df4_folds, df4_test = split_data(df4_cleaned)

In [79]:
df1_train

,wave,relstat,nkids,pinc_decile,hhincnet_decile,empl,edu,ethni,health,bmi_category,...,extraversion,agreeableness,conscientiousness,neuroticism,openness,denomination,rel,abortion,migback,hormon_iudor
61,12,LAT,0,1.0,2.0,Marginal or part-time employment,Advanced,Ethnic-German,Very good,Normal,...,4.00,2.00,3.75,3.75,2.2,Protestant,Never,0.0,1.0,1.0
601,11,Cohabiting,0,3.0,6.0,Full-time or self-employment,Intermediate,German native,Satisfactory,Normal,...,3.25,3.00,3.50,2.75,3.4,Roman Catholic,Seldom,0.0,0.0,0.0
16,11,Cohabiting,0,1.0,7.0,Education/not working,Advanced,Ethnic-German,Very good,Normal,...,4.50,3.75,4.00,3.00,3.2,Protestant,Never,0.0,1.0,1.0
413,12,Cohabiting,0,1.0,3.0,Education/not working,Advanced,German native,Good,Normal,...,2.25,2.25,4.75,2.75,2.4,None,Never,0.0,0.0,1.0
67,10,Cohabiting,0,7.0,8.0,Full-time or self-employment,Intermediate,German native,Not so good,Normal,...,3.75,4.00,3.75,3.00,3.2,None,Never,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,13,LAT,0,5.0,10.0,Full-time or self-employment,Advanced,Turkish background,Very good,Underweight,...,3.75,3.75,3.75,2.00,3.2,Muslim,Never,0.0,1.0,1.0
547,11,Single,0,5.0,3.0,Marginal or part-time employment,Advanced,German native,Good,Obese,...,4.75,3.25,4.25,3.00,2.2,Protestant,Occasionally,0.0,0.0,0.0
275,11,Single,0,2.0,6.0,Marginal or part-time employment,Advanced,German native,Very good,Normal,...,2.25,3.00,3.25,2.75,3.6,Roman Catholic,Never,0.0,0.0,1.0
565,13,Cohabiting,0,4.0,6.0,Full-time or self-employment,Advanced,German native,Not so good,Overweight,...,4.75,2.50,3.50,4.75,4.2,Protestant,Never,0.0,0.0,0.0


In [84]:
def encode_data(df_train, df_test):
    categorical_cols = df_train.select_dtypes(include=['category']).columns

    binary_cols = [col for col in categorical_cols if df_train[col].nunique() == 2]
    binary_numeric_cols = [col for col in binary_cols if set(df_train[col].unique()) == {0, 1}]
    non_binary_cols = list(set(categorical_cols) - set(binary_cols))

    df_train_encoded = df_train.copy()
    df_test_encoded = df_test.copy()

    df_train_encoded[binary_numeric_cols] = df_train_encoded[binary_numeric_cols].astype(int)
    df_test_encoded[binary_numeric_cols] = df_test_encoded[binary_numeric_cols].astype(int)

    df_train_encoded = pd.get_dummies(df_train_encoded, columns=non_binary_cols)
    df_test_encoded = pd.get_dummies(df_test_encoded, columns=non_binary_cols)

    return df_train_encoded, df_test_encoded

df1_train_encoded, df1_test_encoded = encode_data(df1_train, df1_test)

In [85]:
df1_train_encoded

,wave,pinc_decile,hhincnet_decile,val1i3,val1i4,val1i5,urban,sex_often,extraversion,agreeableness,...,addchild_No,addchild_Unsure,addchild_Yes,rel_Frequent,rel_Never,rel_Occasionally,rel_Seldom,empl_Education/not working,empl_Full-time or self-employment,empl_Marginal or part-time employment
61,12,1.0,2.0,3,1,1.0,Urban,1,4.00,2.00,...,False,False,True,False,True,False,False,False,False,True
601,11,3.0,6.0,4,1,4.0,Rural,1,3.25,3.00,...,False,False,True,False,False,False,True,False,True,False
16,11,1.0,7.0,1,1,1.0,Rural,0,4.50,3.75,...,False,False,True,False,True,False,False,True,False,False
413,12,1.0,3.0,3,2,2.0,Urban,1,2.25,2.25,...,False,False,True,False,True,False,False,True,False,False
67,10,7.0,8.0,4,1,3.0,Urban,1,3.75,4.00,...,False,False,True,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,13,5.0,10.0,2,1,4.0,Urban,0,3.75,3.75,...,False,False,True,False,True,False,False,False,True,False
547,11,5.0,3.0,2,1,1.0,Urban,0,4.75,3.25,...,False,False,True,False,False,True,False,False,False,True
275,11,2.0,6.0,2,1,4.0,Rural,0,2.25,3.00,...,False,False,True,False,True,False,False,False,False,True
565,13,4.0,6.0,1,1,4.0,Rural,0,4.75,2.50,...,False,False,True,False,True,False,False,False,True,False


# Supervised Classifier

## Random Forest

In [57]:
# Define the hyperparameters
n_estimators_full = [10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
max_depth_full = list(range(5,21))
min_samples_split_full = list(range(2,41))

# Initialize the model
model_rf = RandomForestClassifier(random_state=666)

# Define the distributions
distributions = dict(n_estimators=n_estimators_full, 
                     max_depth=max_depth_full, 
                     min_samples_split=min_samples_split_full)

# Initialize the RandomizedSearchCV
clf_rf = RandomizedSearchCV(model_rf, distributions, 
                            n_iter=100, verbose=2, n_jobs=4, 
                            scoring='accuracy', refit='accuracy', 
                            cv=5, 
                            random_state=666)

In [58]:
# Fit the model
search = clf_rf.fit(df1_train_encoded.drop('hormon_iudor', axis=1), df1_train_encoded['hormon_iudor'])

# Print the best estimator and score
print(search.best_estimator_)
print(search.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


ValueError: 
All the 500 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\chanho\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\chanho\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\chanho\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_forest.py", line 363, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\chanho\AppData\Roaming\Python\Python311\site-

In [ ]:
# Train the final model
rf_final = RandomForestClassifier(max_depth=search.best_estimator_.max_depth, min_samples_split=search.best_estimator_.min_samples_split, n_estimators=search.best_estimator_.n_estimators, random_state=666)
rf_final.fit(df1_train.drop('hormon_iudor', axis=1), df1_train['hormon_iudor'])

# Make predictions
pred_test_rf = rf_final.predict(df1_test.drop('hormon_iudor', axis=1))

# Print the accuracy and confusion matrix
print(accuracy_score(df1_test['hormon_iudor'], pred_test_rf))
print(confusion_matrix(df1_test['hormon_iudor'], pred_test_rf))


### Select Best Hyperparameters

In [ ]:
# Select the best hyperparameters
best_params_df1 = df1_grid_search_refined.best_params_
best_params_df2 = df2_grid_search_refined.best_params_
best_params_df3 = df3_grid_search_refined.best_params_
best_params_df4 = df4_grid_search_refined.best_params_


## XGBoost

In [ ]:
# Initialize the XGBoost model
model_xgb = GradientBoostingClassifier(random_state=666)

# Define the distributions
distributions_xgb = dict(n_estimators=n_estimators_full, 
                         max_depth=max_depth_full, 
                         min_samples_split=min_samples_split_full)

# Initialize the RandomizedSearchCV
clf_xgb = RandomizedSearchCV(model_xgb, distributions_xgb, 
                             n_iter=100, verbose=2, n_jobs=4, 
                             scoring='accuracy', refit='accuracy', 
                             cv=5, 
                             random_state=666)

# Fit the model
search_xgb = clf_xgb.fit(df1_train_encoded.drop('hormon_iudor', axis=1), df1_train_encoded['hormon_iudor'])

# Print the best estimator and score
print(search_xgb.best_estimator_)

# Train the final model
xgb_final = GradientBoostingClassifier(max_depth=search_xgb.best_estimator_.max_depth, min_samples_split=search_xgb.best_estimator_.min_samples_split, n_estimators=search_xgb.best_estimator_.n_estimators, random_state=666)

xgb_final.fit(df1_train.drop('hormon_iudor', axis=1), df1_train['hormon_iudor'])

# Make predictions
pred_test_xgb = xgb_final.predict(df1_test.drop('hormon_iudor', axis=1))